## TCC CBB

Desenvolvido por Ricardo e Thyago.

#### Objetivo: 

Aplicação das técnicas de classificação SVM e XGboost junto ao algoritmo bio-inspirado FA.

Neste script é aplicado os algoritmos de classificação SVM e XGboost também o modelo estatístico
com seu ponderamento feito pelos algoritmos Bio-inspirados.
Também é calculado a Média, tempo e desvio-padrão, bem como o Tempo de execução e a Distância euclidiana 
entre a posição dos melhores agentes para o FA.

Fez-se a analise dos parâmetros de aplicação do XGBoost e a técnica foi reaplicada com os paramêtros atualizados.

### Import de bibliotecas

In [ ]:
import random
import bigml.api
import os
import pandas as pd
from bigml.api import BigML
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import plotly.express as px
import cufflinks as cf

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['animation.ffmpeg_path'] = 'C:\\Users\\Ricardo\\Desktop\\ffmpeg-20200206-343ccfc-win64-static\\bin\\ffmpeg.exe'

### Import das bibliotecas do XGBoost

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier

In [ ]:
api = BigML('ricardomorellosantos','b34ec3c18161b1da38b0c5e04520224f7544405e')

In [ ]:
api.download_dataset(dataset='dataset/5e356cd41efc9271bf006ea2', filename=os.getcwd()+'\\bigml.csv')

### Funções de tratamento da base de dados Churn in Telecom's data set

In [ ]:
def converte_binario(palavra):
    if palavra=='Yes' or palavra==True:
        return 0
    elif palavra=='No' or palavra==False:
        return 1

In [ ]:
def descarregaBaseDados():
    df_bigml = pd.read_csv(os.getcwd()+'\\bigml.csv')
    states = df_bigml['State'].value_counts()
    df_bigml['Voice mail plan'] = df_bigml['Voice mail plan'].apply(converte_binario)
    df_bigml['International plan'] = df_bigml['International plan'].apply(converte_binario)
    df_bigml['Churn'] = df_bigml['Churn'].apply(converte_binario)
    df_bigml_target = df_bigml['Churn']
    df_bigml.drop('Churn', axis=1, inplace=True)
    df_bigml_target.head()
    print(df_bigml_target.value_counts())
    array_estados = []
    i = 0
    for index, val in states.iteritems():
        array_estados.append(index)    
        i = 0
    for estado in array_estados:
        df_bigml['State'] = df_bigml['State'].replace(to_replace=estado, value=i)
        i = i+1
    return df_bigml, df_bigml_target

### Funções para Kfold, XGBoost e SVM

In [ ]:
def aplicaKFold(x_pca, df_bigml_target):
    divisao = 0.2
    X_train, X_test, y_train, y_test = train_test_split(x_pca, df_bigml_target, test_size=divisao, random_state = 42)
    return (X_train, X_test, y_train, y_test)

In [ ]:
def aplicaSVM(X_train, X_test, y_train): 
    modelSVM = SVC(probability=True, kernel = 'rbf')
    modelSVM.fit(X_train, y_train)
    predictionsSVM = modelSVM.predict_proba(X_test)

    return predictionsSVM

In [ ]:
def aplicaXGBoost(X_train, X_test, y_train):
    xg_reg = xgb.XGBClassifier(objective ='binary:logistic', colsample_bytree = 0.3, learning_rate = 1,
                max_depth = 10, alpha = 10, n_estimators = 10)
    xg_reg.fit(X_train,y_train)
    predictionsXGBoost = xg_reg.predict_proba(X_test)
    return predictionsXGBoost

In [ ]:
###

In [ ]:
class sw(object):

    def __init__(self):

        self.__Positions = []
        self.__Gbest = []

    def _set_Gbest(self, Gbest):
        self.__Gbest = Gbest

    def _points(self, agents):
        self.__Positions.append([list(i) for i in agents])

    def get_agents(self):
        """Returns a history of all agents of the algorithm (return type:
        list)"""

        return self.__Positions

    def get_Gbest(self):
        """Return the best position of algorithm (return type: list)"""

        return list(self.__Gbest)

In [ ]:
%matplotlib inline

In [ ]:
from matplotlib import pyplot as plt
import matplotlib.animation
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
import matplotlib.animation as animation

def animationFA(agents, function, lb, ub,y_test, sr=False):

    side = np.linspace(lb, ub, (ub - lb) * 5)
    X, Y = np.meshgrid(side, side)
    Z = np.array([np.array([function([X[i][j], Y[i][j]], y_test)
                            for j in range(len(X))])
                  for i in range(len(X[0]))])

    fig, ax = plt.subplots(1, 1, figsize=(12,6))
    ax.set_xlabel('SVM')
    ax.set_ylabel('XGBoost')
    plt.axes(xlim=(lb, ub), ylim=(lb, ub))
    plt.pcolormesh(X, Y, Z, shading='flat')
    plt.colorbar()
    x = np.array([j[0] for j in agents[0]])
    y = np.array([j[1] for j in agents[0]])
    sc = plt.scatter(x, y, color='black')

    plt.title(function.__name__, loc='left')
    plt
    def an(i):
        x = np.array([j[0] for j in agents[i]])
        y = np.array([j[1] for j in agents[i]])
        sc.set_offsets(list(zip(x, y)))
        plt.title('iteration: {}'.format(i), loc='right')
    
    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)
    ani = matplotlib.animation.FuncAnimation(fig, an, frames=len(agents) - 1)

    if sr:
        ani.save('im.mp4', writer=writer)

    plt.show()


def animation3D(agents, function, lb, ub, y_test, sr=False):

    side = np.linspace(lb, ub, 45)
    X, Y = np.meshgrid(side, side)
    zs = np.array([function([x, y], y_test) for x, y in zip(np.ravel(X), np.ravel(Y))])
    Z = zs.reshape(X.shape)

    fig = plt.figure()

    ax = Axes3D(fig)
    surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap='jet',
                           linewidth=0, antialiased=False)
    ax.set_xlim(lb, ub)
    ax.set_ylim(lb, ub)

    ax.zaxis.set_major_locator(LinearLocator(10))
    ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

    fig.colorbar(surf, shrink=0.5, aspect=5)

    iter = len(agents)
    n = len(agents[0])
    t = np.array([np.ones(n) * i for i in range(iter)]).flatten()
    b = []
    [[b.append(agent) for agent in epoch] for epoch in agents]
    c = [function(x, y_test) for x in b]
    a = np.asarray(b)
    df = pd.DataFrame({"time": t, "x": a[:, 0], "y": a[:, 1], "z": c})

    def update_graph(num):
        data = df[df['time'] == num]
        graph._offsets3d = (data.x, data.y, data.z)
        title.set_text(function.__name__ + " " * 45 + 'iteration: {}'.format(
            num))

    title = ax.set_title(function.__name__ + " " * 45 + 'iteration: 0')

    data = df[df['time'] == 0]
    graph = ax.scatter(data.x, data.y, data.z, color='black')

    ani = matplotlib.animation.FuncAnimation(fig, update_graph, iter,
                                             interval=50, blit=False)

    if sr:

        ani.save('result.mp4')

    plt.show()

In [ ]:
def trunca_agentes(array):
    for line in array:
        if line[0] + line[1] > 1:
            line[0] = line[0]/2
            line[1] = line[1]/2 
    return array;

### Classe para o algoritmo Firefly FA

In [ ]:
from math import exp
import numpy as np

class fa(sw):
    """
    Firefly Algorithm
    """
    def __init__(self, n, function, lb, ub, dimension, iteration, y_test, vetor_pbest, vetor_acertos, csi=1, psi=1,
                 alpha0=10, alpha1=0.5, norm0=0, norm1=0.1):
        """
        :param n: number of agents
        :param function: test function
        :param lb: lower limits for plot axes
        :param ub: upper limits for plot axes
        :param dimension: space dimension
        :param iteration: number of iterations
        :param csi: mutual attraction (default value is 1)
        :param psi: light absorption coefficient of the medium
        (default value is 1)
        :param alpha0: initial value of the free randomization parameter alpha
        (default value is 1)
        :param alpha1: final value of the free randomization parameter alpha
        (default value is 0.1)
        :param norm0: first parameter for a normal (Gaussian) distribution
        (default value is 0)
        :param norm1: second parameter for a normal (Gaussian) distribution
        (default value is 0.1)
        """

        super(fa, self).__init__()

        self.__agents = trunca_agentes(np.random.uniform(lb, ub, (n, dimension)))
        self._points(self.__agents)
        Pbest = self.__agents[np.array([function(x, y_test)
                                        for x in self.__agents]).argmax()]
        Gbest = Pbest
        vetor_pbest.append(Pbest)
        vetor_acertos.append(function(Pbest, y_test))
        for t in range(iteration):
            alpha = alpha1 + (alpha0 - alpha1) * exp(-t)
            for i in range(n):
                fitness = [function(x, y_test) for x in self.__agents]
                for j in range(n):
                    if fitness[i] < fitness[j]:
                        self.__move(i, j, t, csi, psi, alpha, dimension,
                                    norm0, norm1)
                    else:
                        self.__agents[i] += np.random.normal(norm0, norm1,
                                                             dimension)

            self.__agents = trunca_agentes(np.clip(self.__agents, lb, ub))
            self._points(self.__agents)
            
            Pbest = self.__agents[
                np.array([function(x, y_test) for x in self.__agents]).argmax()]
            if function(Pbest, y_test) > function(Gbest, y_test):
                Gbest = Pbest
            vetor_pbest.append(Gbest)
            vetor_acertos.append(function(Gbest, y_test))
        self._set_Gbest(Gbest)

    def __move(self, i, j, t, csi, psi, alpha, dimension, norm0, norm1):
        r = np.linalg.norm(self.__agents[i] - self.__agents[j])
        beta = csi / (1 + psi * r ** 2)
        self.__agents[i] = self.__agents[j] + beta * (
            self.__agents[i] - self.__agents[j]) + alpha * exp(-t) * \
                                                   np.random.normal(norm0,
                                                                    norm1,
                                                                    dimension)

### Função para a verificaçao de acertos do modelo estatístico

In [ ]:
def verifica_acertos(predicoes, y_test):
    qtd_acertos = []
    for i in range(predicoes.shape[0]):
        if predicoes[i] == y_test[i]:
            qtd_acertos.append(1)
        else:
            qtd_acertos.append(0)
    return np.asarray(qtd_acertos)

In [ ]:
def retorna_acertos(x, y_test):
    svm_alpha = ( ( 1 - predictionsSVM[:,1] ) ** x[0] )
    xgboost_beta =  ( ( 1 - predictionsXGBoost[:,1] ) ** x[1] )
    predicoes = 1 - (svm_alpha*xgboost_beta)
    qtd_acertos = np.subtract(y_test,np.round(predicoes))
    return np.count_nonzero(qtd_acertos == 0)

### Aplicação do XGBoost e SVM junto ao algoritmo Firefly FA

In [ ]:
import SwarmPackagePy
import matplotlib.pyplot as plt
from SwarmPackagePy import testFunctions as tf

df_bigml, df_bigml_target = descarregaBaseDados()
scaler = StandardScaler()

scaler.fit(df_bigml)

scaled_data = scaler.transform(df_bigml)  
pca_svm = PCA(n_components = 16)
pca_svm.fit(scaled_data)
x_pca_svm = pca_svm.transform(scaled_data)

pca_xgboost = PCA(n_components=17)
pca_xgboost.fit(scaled_data)
x_pca_xgboost = pca_xgboost.transform(scaled_data)

X_train_svm, X_test_svm, y_train_svm, y_test_svm = aplicaKFold(x_pca_svm, df_bigml_target)
X_train_xgboost, X_test_xgboost, y_train_xgboost, y_test_xgboost = aplicaKFold(x_pca_xgboost, df_bigml_target)

predictionsSVM = aplicaSVM(X_train_svm, X_test_svm, y_train_svm)
predictionsXGBoost = aplicaXGBoost(X_train_xgboost, X_test_xgboost, y_train_xgboost)

vetor_pbest = []
vetor_acertos = []

alh = fa(40, retorna_acertos, 0, 1, 2, 50, y_test_svm, vetor_pbest, vetor_acertos)
arr = np.array(alh.get_agents())
animationFA(alh.get_agents(), retorna_acertos, 0, 1, y_test_svm, sr=True)
animation3D(alh.get_agents(), retorna_acertos, 0, 1, y_test_svm, sr=True)
print(alh.get_Gbest())

### APlicação do Modelo Estatístico com o ponderamento dos melhores agentes encontrados na sa´da do algoritmos FA

In [ ]:
svm_alpha = ( ( 1 - predictionsSVM[:,1] ) ** alh.get_Gbest()[0] )
xgboost_beta =  ( ( 1 - predictionsXGBoost[:,1] ) ** alh.get_Gbest()[1])
predicoes = 1 - (svm_alpha*xgboost_beta)
qtd_acertos = np.subtract(y_test_svm,np.round(predicoes))
np.count_nonzero(qtd_acertos == 0)/len(y_test_svm) * 100

### Curva ROC

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
fpr, tpr, thresholds = roc_curve(y_test_svm, predicoes)
roc_auc = auc(fpr, tpr)
print(roc_auc)
fig = plt.figure(figsize=(12,6))
plt.xlabel('1-Especificidade')
plt.ylabel('Sensibilidade')
plt.plot([0, 1], [0, 1], color='red', linestyle='--')
plt.title('Curva ROC Modelo estatístico')
plt.plot(fpr, tpr, color='blue', lw=2, label='ROC area = %0.2f)' % roc_auc)
plt.legend(loc="lower right")
plt.show()
fig.savefig('ROC_Modelo_Estatistico_BIGML_SVM.PNG')

In [ ]:
vetor_acertos
assertividade = [(x / 667) * 100 for x in vetor_acertos]

In [ ]:
df_acertos = pd.DataFrame(assertividade)

In [ ]:
fig = plt.figure(figsize=(12,8))
plt.rcParams.update({'font.size': 20})
sns.lineplot(data=df_acertos)
plt.legend('')
plt.title('Porcentagem de acertos FA')
plt.xlabel('Iteração')
plt.ylabel('Porcentagem')
plt.savefig('bigml_fa_porcentagem_acertos.png')
plt.show()


In [ ]:
df_pbest = pd.DataFrame(vetor_pbest, columns = ['x', 'y'])
df_pbest['iteracao'] = np.arange(0, 51)

# Média, tempo e desvio-padrão FA

In [ ]:
from time import perf_counter 
  

In [ ]:
import SwarmPackagePy
import matplotlib.pyplot as plt
from SwarmPackagePy import testFunctions as tf


df_bigml, df_bigml_target = descarregaBaseDados()
scaler = StandardScaler()

scaler.fit(df_bigml)

tempo = []
melhor = []

for i in range(50):
    print(i)
    t1_start = perf_counter()  
    scaled_data = scaler.transform(df_bigml)  
    pca_svm = PCA(n_components = 16)
    pca_svm.fit(scaled_data)
    x_pca_svm = pca_svm.transform(scaled_data)

    pca_xgboost = PCA(n_components=17)
    pca_xgboost.fit(scaled_data)
    x_pca_xgboost = pca_xgboost.transform(scaled_data)

    X_train_svm, X_test_svm, y_train_svm, y_test_svm = aplicaKFold(x_pca_svm, df_bigml_target)
    X_train_xgboost, X_test_xgboost, y_train_xgboost, y_test_xgboost = aplicaKFold(x_pca_xgboost, df_bigml_target)

    predictionsSVM = aplicaSVM(X_train_svm, X_test_svm, y_train_svm)
    predictionsXGBoost = aplicaXGBoost(X_train_xgboost, X_test_xgboost, y_train_xgboost)

    vetor_pbest = []
    vetor_acertos = []

    alh = fa(40, retorna_acertos, 0, 1, 2, 50, y_test_svm, vetor_pbest, vetor_acertos)
    arr = np.array(alh.get_agents())
    t1_stop = perf_counter() 
    melhor.append(alh.get_Gbest())
    tempo.append(t1_stop-t1_start)
    #animationFA(alh.get_agents(), retorna_acertos, 0, 1, y_test_svm, sr=True)
    #animation3D(alh.get_agents(), retorna_acertos, 0, 1, y_test_svm, sr=True)
    print(alh.get_Gbest())

In [ ]:
df_media_desvpad_fa = pd.DataFrame()
df_media_desvpad_fa['tempo'] = tempo

In [ ]:
df_media_desvpad_fa['solucoes'] = melhor

In [ ]:
auc_fa = []
for solucao in melhor:
    svm_alpha = ( ( 1 - predictionsSVM[:,1] ) ** solucao[0] )
    xgboost_beta =  ( ( 1 - predictionsXGBoost[:,1] ) ** solucao[1])
    predicoes = 1 - (svm_alpha*xgboost_beta)
    qtd_acertos = np.subtract(y_test_svm,np.round(predicoes))
    np.count_nonzero(qtd_acertos == 0)/len(y_test_svm) * 100
    fpr, tpr, thresholds = roc_curve(y_test_svm, predicoes)
    auc_fa.append(auc(fpr, tpr))

In [ ]:
df_media_desvpad_fa['aucs'] = auc_fa

In [ ]:
df_media_desvpad_fa

In [ ]:
df_media_desvpad_fa[['solucao_x','solucao_y']] = pd.DataFrame(df_media_desvpad_fa.solucoes.values.tolist(), index= df_media_desvpad_fa.index)

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.title('Tempo de execução do algoritmo FA com random_state estático')
plt.xlabel('Execução')
plt.ylabel('Tempo (s)')
df_media_desvpad_fa['tempo'].plot(yerr=df_media_desvpad_fa['tempo'].std(), marker = 'o')
plt.savefig('tempo_execucao_fa_bigml_estatico.png')



### Cálculo da distância Euclidiana entre as posições dos melhores agentes encontrados no FA

In [ ]:
from scipy.spatial import distance
distancias_euclidianas = []
cont = 0
for i in df_media_desvpad_fa['solucoes']:
    if cont < len(df_media_desvpad_fa['solucoes']) - 1:
        proximo = df_media_desvpad_fa['solucoes'][cont+1]
        primeiro_ponto = (i[0], i[1])
        segundo_ponto = (proximo[0], proximo[1])
        distancias_euclidianas.append(distance.euclidean(primeiro_ponto, segundo_ponto))
        cont = cont+1
    else:
        distancias_euclidianas.append(0)

In [ ]:
df_media_desvpad_fa['distancias_euclidianas'] = distancias_euclidianas

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.title('Distância euclidiana entre a posição dos melhores agentes com random_state estático')
plt.rcParams.update({'font.size': 18})
plt.xlabel('Execução')
plt.ylabel('Distância euclidiana')
df_media_desvpad_fa['distancias_euclidianas'].plot(yerr=df_media_desvpad_fa['distancias_euclidianas'].std(), marker = 'o', color='red')
plt.savefig('distancia_euclidiana_fa_bigml_estatico.png')



### AUCs nas iterações do algoritmo FA

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.title('AUCs nas execuções com random_state estático')
plt.rcParams.update({'font.size': 18})
plt.xlabel('Execução')
plt.ylabel('AUC')
df_media_desvpad_fa['aucs'] = df_media_desvpad_fa['aucs']
print(df_media_desvpad_fa['aucs'].std())
df_media_desvpad_fa['aucs'].plot(yerr=df_media_desvpad_fa['aucs'].std(), marker = 'o', color='green')
plt.savefig('auc_fa_bigml_estatico.png')


# Média, tempo e desvio-padrão FA com diferentes amostras da base até a convergência

In [ ]:
def aplicaKFoldRand(x_pca, df_bigml_target, rand):
    divisao = 0.2
    X_train, X_test, y_train, y_test = train_test_split(x_pca, df_bigml_target, test_size=divisao, random_state = rand)
    return (X_train, X_test, y_train, y_test)

In [ ]:
import SwarmPackagePy
import matplotlib.pyplot as plt
from SwarmPackagePy import testFunctions as tf
from sklearn.naive_bayes import GaussianNB


gnb = GaussianNB()

df_bigml, df_bigml_target = descarregaBaseDados()
scaler = StandardScaler()

scaler.fit(df_bigml)

tempo_rand = []
melhor_rand = []
rands = []
auc_fa = []
auc_svm = []
auc_xgboost = []
convergencia = False
media_aucs = 0
auc_anterior = 0
total_iguais = 0
i = 0
while(not convergencia):
    print(i)
    rand = np.random.randint(0, 1000)
    t1_start = perf_counter()  
    scaled_data = scaler.transform(df_bigml)  
    pca_svm = PCA(n_components = 16)
    pca_svm.fit(scaled_data)
    x_pca_svm = pca_svm.transform(scaled_data)

    pca_xgboost = PCA(n_components=17)
    pca_xgboost.fit(scaled_data)
    x_pca_xgboost = pca_xgboost.transform(scaled_data)
    rands.append(rand)
    X_train_svm, X_test_svm, y_train_svm, y_test_svm = aplicaKFoldRand(x_pca_svm, df_bigml_target, rand)
    X_train_xgboost, X_test_xgboost, y_train_xgboost, y_test_xgboost = aplicaKFoldRand(x_pca_xgboost, df_bigml_target, rand)
    
    predictionsSVM = aplicaSVM(X_train_svm, X_test_svm, y_train_svm)
    predictionsXGBoost = aplicaXGBoost(X_train_xgboost, X_test_xgboost, y_train_xgboost)
    
    fprs, tprs, thresholdss = roc_curve(y_test_svm, predictionsSVM[:,1])
    fprx, tprx, thresholdsx = roc_curve(y_test_svm, predictionsXGBoost[:,1])
    
    auc_svm.append(auc(fprs, tprs))
    auc_xgboost.append(auc(fprx, tprx))
    
    vetor_pbest = []
    vetor_acertos = []

    alh = fa(40, retorna_acertos, 0, 1, 2, 50, y_test_svm, vetor_pbest, vetor_acertos)
    solucao = alh.get_Gbest()
    svm_alpha = ( ( 1 - predictionsSVM[:,1] ) ** solucao[0] )
    xgboost_beta =  ( ( 1 - predictionsXGBoost[:,1] ) ** solucao[1])
    predicoes = 1 - (svm_alpha*xgboost_beta)
    qtd_acertos = np.subtract(y_test_svm,np.round(predicoes))
    np.count_nonzero(qtd_acertos == 0)/len(y_test_svm) * 100
    fpr, tpr, thresholds = roc_curve(y_test_svm, predicoes)
    auc_fa.append(auc(fpr, tpr))
    
    media_aucs = round(sum(auc_fa)/len(auc_fa), 2)
    if media_aucs == auc_anterior:
        total_iguais = total_iguais + 1
    else:
        total_iguais = 0
    
    if total_iguais == 10:
        convergencia = True
    
    print('media_aucs:{}'.format(media_aucs))
    auc_anterior = media_aucs
    
    arr = np.array(alh.get_agents())
    t1_stop = perf_counter() 
    melhor_rand.append(alh.get_Gbest())
    tempo_rand.append(t1_stop-t1_start)
    
    print(alh.get_Gbest())

In [ ]:
df_media_desvpad_fa_rand = pd.DataFrame()
df_media_desvpad_fa_rand['tempo'] = tempo_rand
df_media_desvpad_fa_rand['solucoes'] = melhor_rand
df_media_desvpad_fa_rand['rand_values'] = rands
df_media_desvpad_fa_rand['aucs'] = auc_fa
df_media_desvpad_fa_rand['auc_svm'] = auc_svm
df_media_desvpad_fa_rand['auc_xgboost'] = auc_xgboost
df_media_desvpad_fa_rand[['solucao_x','solucao_y']] = pd.DataFrame(df_media_desvpad_fa_rand.solucoes.values.tolist(), index= df_media_desvpad_fa_rand.index)

In [ ]:
print(df_media_desvpad_fa_rand['tempo'].std())
fig = plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 16})
plt.title('Tempo de execução até convergência da média')
plt.xlabel('Execução')
plt.ylabel('Tempo (s)')
df_media_desvpad_fa_rand['tempo'].plot(yerr=df_media_desvpad_fa_rand['tempo'].std(), marker = 'o')
plt.savefig('tempo_execucao_fa_bigml.png')

In [ ]:
from scipy.spatial import distance
distancias_euclidianas = []
cont = 0
for i in df_media_desvpad_fa_rand['solucoes']:
    if cont < len(df_media_desvpad_fa_rand['solucoes']) - 1:
        proximo = df_media_desvpad_fa_rand['solucoes'][cont+1]
        primeiro_ponto = (i[0], i[1])
        segundo_ponto = (proximo[0], proximo[1])
        distancias_euclidianas.append(distance.euclidean(primeiro_ponto, segundo_ponto))
        cont = cont+1
    else:
        distancias_euclidianas.append(0)

In [ ]:
df_media_desvpad_fa_rand['distancias_euclidianas'] = distancias_euclidianas

In [ ]:
df_media_desvpad_fa_rand['distancias_euclidianas'].std()

In [ ]:
df_media_desvpad_fa_rand['AUCs modelo estatístico'].mean()

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.title('Distância euclidiana entre os melhores agentes até convergência da média')
plt.rcParams.update({'font.size': 13})
plt.xlabel('Execução')
plt.ylabel('Distância euclidiana')
df_media_desvpad_fa_rand['distancias_euclidianas'] = np.clip(df_media_desvpad_fa_rand['distancias_euclidianas'],0, 1)
df_media_desvpad_fa_rand['distancias_euclidianas'].plot(yerr=df_media_desvpad_fa_rand['distancias_euclidianas'].std(), marker = 'o', color='red')
plt.savefig('distancia_euclidiana_fa_bigml.png')
print(df_media_desvpad_fa_rand['distancias_euclidianas'].std())
print(df_media_desvpad_fa_rand['distancias_euclidianas'])

In [ ]:
import matplotlib as mpl

# create valid markers from mpl.markers
valid_markers = ([item[0] for item in mpl.markers.MarkerStyle.markers.items() if
item[1] is not 'nothing' and not item[1].startswith('tick') and not item[1].startswith('caret')])
df_media_desvpad_fa_rand.rename(columns = {'aucs':'AUCs modelo estatístico', 'auc_svm':'AUCs SVM', 'auc_xgboost' : 'AUCs XGBoost'}, inplace = True) 

ax = df_media_desvpad_fa_rand[['AUCs modelo estatístico', 'AUCs SVM', 'AUCs XGBoost']].plot(figsize=(16,10))


# valid_markers = mpl.markers.MarkerStyle.filled_markers
markers = np.random.choice(valid_markers, df_media_desvpad_fa_rand.shape[1], replace=False)
for i, line in enumerate(ax.get_lines()):
    line.set_marker(markers[i])

plt.title('AUCs nas execuções até convergência da média')
plt.rcParams.update({'font.size': 18})
plt.xlabel('Execução')
plt.ylabel('AUC')


plt.savefig('auc_fa_bigml.png')

print(df_media_desvpad_fa_rand['AUCs modelo estatístico'].std())

## Aplicações do algoritmo bio-inspirado PSO

### Classe PSO

In [ ]:
class swa(object):

    def __init__(self):

        self.__Positions = []
        self.__Gbest = []

    def _set_Gbest(self, __Gbest):
        self.__Gbest = __Gbest

    def _points(self, agents):
        self.__Positions.append([list(i) for i in agents])

    def get_agents(self):
        """Returns a history of all agents of the algorithm (return type:
        list)"""

        return self.__Positions

    def get_Gbest(self):
        """Return the best position of algorithm (return type: list)"""

        return list(self.__Gbest)

In [ ]:
import numpy as np

class pso(swa):
    """
    Particle Swarm Optimization
    """

    def __init__(self, n, function, lb, ub, dimension, iteration, y_test_svm,vetor_pbest, vetor_acertos, w=0.5, c1=1,
                 c2=1):
        """
        :param n: number of agents
        :param function: test function
        :param lb: lower limits for plot axes
        :param ub: upper limits for plot azes
        :param dimension: space dimension
        :param iteration: the number of iterations
        :param w: balance between the range of research and consideration for
        suboptimal decisions found (default value is 0.5):
        w>1 the particle velocity increases, they fly apart and inspect
         the space more carefully;
        w<1 particle velocity decreases, convergence speed depends
        on parameters c1 and c2 ;
        :param c1: ratio between "cognitive" and "social" component
        (default value is 1)
        :param c2: ratio between "cognitive" and "social" component
        (default value is 1)
        """

        super(pso, self).__init__()

        self.__agents = trunca_agentes(np.random.uniform(lb, ub, (n, dimension)))
        velocity = np.zeros((n, dimension))
        self._points(self.__agents)

        Pbest = self.__agents[np.array([function(x, y_test_svm)
                                        for x in self.__agents]).argmax()]
        Gbest = Pbest
        vetor_pbest.append(Pbest)
        vetor_acertos.append(function(Pbest, y_test_svm))
        
        for t in range(iteration):
            r1 = np.random.random((n, dimension))
            r2 = np.random.random((n, dimension))
            velocity = w * velocity + c1 * r1 * (
                Pbest - self.__agents) + c2 * r2 * (
                Gbest - self.__agents)
            self.__agents += velocity
            self.__agents = trunca_agentes(np.clip(self.__agents, lb, ub))
            self._points(self.__agents)
         
            Pbest = self.__agents[
                np.array([function(x, y_test_svm) for x in self.__agents]).argmax()]
            if function(Pbest, y_test_svm) > function(Gbest, y_test_svm):
                Gbest = Pbest
            vetor_pbest.append(Pbest)
            vetor_acertos.append(function(Pbest, y_test_svm))
        
        self._set_Gbest(Gbest)
        print(Gbest)

## Aplicação dos algoritmos XGBoost e SVM junto ao algoritmos PSO até sua convergência 

In [ ]:
import SwarmPackagePy
import matplotlib.pyplot as plt
from SwarmPackagePy import testFunctions as tf
from sklearn.naive_bayes import GaussianNB


gnb = GaussianNB()

df_bigml, df_bigml_target = descarregaBaseDados()
scaler = StandardScaler()

scaler.fit(df_bigml)

tempo_rand = []
melhor_rand = []
rands = []
auc_fa = []
auc_svm = []
auc_xgboost = []
convergencia = False
media_aucs = 0
auc_anterior = 0
total_iguais = 0
i = 0
while(not convergencia):
    print(i)
    rand = np.random.randint(0, 1000)
    t1_start = perf_counter()  
    scaled_data = scaler.transform(df_bigml)  
    pca_svm = PCA(n_components = 16)
    pca_svm.fit(scaled_data)
    x_pca_svm = pca_svm.transform(scaled_data)

    pca_xgboost = PCA(n_components=17)
    pca_xgboost.fit(scaled_data)
    x_pca_xgboost = pca_xgboost.transform(scaled_data)
    rands.append(rand)
    X_train_svm, X_test_svm, y_train_svm, y_test_svm = aplicaKFoldRand(x_pca_svm, df_bigml_target, rand)
    X_train_xgboost, X_test_xgboost, y_train_xgboost, y_test_xgboost = aplicaKFoldRand(x_pca_xgboost, df_bigml_target, rand)
    
    predictionsSVM = aplicaSVM(X_train_svm, X_test_svm, y_train_svm)
    predictionsXGBoost = aplicaXGBoost(X_train_xgboost, X_test_xgboost, y_train_xgboost)
    
    fprs, tprs, thresholdss = roc_curve(y_test_svm, predictionsSVM[:,1])
    fprx, tprx, thresholdsx = roc_curve(y_test_svm, predictionsXGBoost[:,1])
    
    auc_svm.append(auc(fprs, tprs))
    auc_xgboost.append(auc(fprx, tprx))
    
    vetor_pbest = []
    vetor_acertos = []

    alh = pso(40, retorna_acertos, 0, 1, 2, 50, y_test_svm, vetor_pbest, vetor_acertos)
    solucao = alh.get_Gbest()
    svm_alpha = ( ( 1 - predictionsSVM[:,1] ) ** solucao[0] )
    xgboost_beta =  ( ( 1 - predictionsXGBoost[:,1] ) ** solucao[1])
    predicoes = 1 - (svm_alpha*xgboost_beta)
    qtd_acertos = np.subtract(y_test_svm,np.round(predicoes))
    np.count_nonzero(qtd_acertos == 0)/len(y_test_svm) * 100
    fpr, tpr, thresholds = roc_curve(y_test_svm, predicoes)
    auc_fa.append(auc(fpr, tpr))
    
    media_aucs = round(sum(auc_fa)/len(auc_fa), 2)
    if media_aucs == auc_anterior:
        total_iguais = total_iguais + 1
    else:
        total_iguais = 0
    
    if total_iguais == 10:
        convergencia = True
    
    print('media_aucs:{}'.format(media_aucs))
    auc_anterior = media_aucs
    
    arr = np.array(alh.get_agents())
    t1_stop = perf_counter() 
    melhor_rand.append(alh.get_Gbest())
    tempo_rand.append(t1_stop-t1_start)
    
    print(alh.get_Gbest())

### Média, desvio padrão para o PSO

In [ ]:
df_media_desvpad_PSO_rand = pd.DataFrame()
df_media_desvpad_PSO_rand['tempo'] = tempo_rand
df_media_desvpad_PSO_rand['solucoes'] = melhor_rand
df_media_desvpad_PSO_rand['rand_values'] = rands
df_media_desvpad_PSO_rand['aucs'] = auc_fa
df_media_desvpad_PSO_rand['auc_svm'] = auc_svm
df_media_desvpad_PSO_rand['auc_xgboost'] = auc_xgboost
df_media_desvpad_PSO_rand[['solucao_x','solucao_y']] = pd.DataFrame(df_media_desvpad_PSO_rand.solucoes.values.tolist(), index= df_media_desvpad_PSO_rand.index)

### Gráfico do tempo de execução do PSO até sua convergência

In [ ]:
print(df_media_desvpad_PSO_rand['tempo'].std())
fig = plt.figure(figsize=(12,6))
plt.rcParams.update({'font.size': 16})
plt.title('Tempo de execução até convergência da média')
plt.xlabel('Execução')
plt.ylabel('Tempo (s)')
df_media_desvpad_PSO_rand['tempo'].plot(yerr=df_media_desvpad_PSO_rand['tempo'].std(), marker = 'o')
plt.savefig('tempo_execucao_pso_bigml.png')

In [ ]:
df_media_desvpad_PSO_rand

### Cálculo da distância euclidiana dos melhores agentes para o PSO

In [ ]:
from scipy.spatial import distance
distancias_euclidianas = []
cont = 0
for i in df_media_desvpad_fa_rand['solucoes']:
    if cont < len(df_media_desvpad_fa_rand['solucoes']) - 1:
        proximo = df_media_desvpad_fa_rand['solucoes'][cont+1]
        primeiro_ponto = (i[0], i[1])
        segundo_ponto = (proximo[0], proximo[1])
        distancias_euclidianas.append(distance.euclidean(primeiro_ponto, segundo_ponto))
        cont = cont+1
    else:
        distancias_euclidianas.append(0)
df_media_desvpad_fa_rand['distancias_euclidianas'] = distancias_euclidianas
fig = plt.figure(figsize=(12,6))
plt.title('Distância euclidiana entre os melhores agentes até convergência da média')
plt.rcParams.update({'font.size': 13})
plt.xlabel('Execução')
plt.ylabel('Distância euclidiana')
df_media_desvpad_fa_rand['distancias_euclidianas'] = np.clip(df_media_desvpad_fa_rand['distancias_euclidianas'],0, 1)
df_media_desvpad_fa_rand['distancias_euclidianas'].plot(yerr=df_media_desvpad_fa_rand['distancias_euclidianas'].std(), marker = 'o', color='red')
plt.savefig('distancia_euclidiana_pso_bigml.png')

print(df_media_desvpad_fa_rand['distancias_euclidianas'].std())

### Gráfico das AUCs em cada execução do PSO até sua convergência

In [ ]:
import matplotlib as mpl

# create valid markers from mpl.markers
valid_markers = ([item[0] for item in mpl.markers.MarkerStyle.markers.items() if
item[1] is not 'nothing' and not item[1].startswith('tick') and not item[1].startswith('caret')])
df_media_desvpad_fa_rand.rename(columns = {'aucs':'AUCs modelo estatístico', 'auc_svm':'AUCs SVM', 'auc_xgboost' : 'AUCs XGBoost'}, inplace = True) 

print(df_media_desvpad_fa_rand['AUCs modelo estatístico'].mean())
ax = df_media_desvpad_fa_rand[['AUCs modelo estatístico', 'AUCs SVM', 'AUCs XGBoost']].plot(figsize=(16,10))


# valid_markers = mpl.markers.MarkerStyle.filled_markers
markers = np.random.choice(valid_markers, df_media_desvpad_fa_rand.shape[1], replace=False)
for i, line in enumerate(ax.get_lines()):
    line.set_marker(markers[i])

plt.title('AUCs nas execuções até convergência')
plt.rcParams.update({'font.size': 18})
plt.xlabel('Execução')
plt.ylabel('AUC')


plt.savefig('auc_pso_bigml.png')


### Análise da avaliação de, os rótulos classificados pelo XGBoost, serem um subconjunto da classificação de rótulos do SVM

In [ ]:
df_analise_subconjunto = pd.DataFrame(data = np.round(predictionsSVM)[:,1], columns = ['SVM'] )

In [ ]:
df_analise_subconjunto['Target'] = y_test_svm.values

In [ ]:
df_analise_subconjunto['XGBoost'] = np.round(predictionsXGBoost[:,1])

In [ ]:
df_analise_subconjunto['Acertos_SVM'] = df_analise_subconjunto['SVM'][(df_analise_subconjunto['SVM'] == df_analise_subconjunto['Target'])].replace(to_replace = 0, value=1)

In [ ]:
df_analise_subconjunto['Acertos_SVM'] = np.nan_to_num(df_analise_subconjunto['Acertos_SVM'])

In [ ]:
df_analise_subconjunto['Acertos_SVM'].value_counts()

In [ ]:
df_analise_subconjunto['Acertos_XGBoost'] = df_analise_subconjunto['XGBoost'][(np.logical_and(df_analise_subconjunto['XGBoost'],1) & np.logical_and(df_analise_subconjunto['Target'],1))].replace(to_replace = 0, value=1)

In [ ]:
df_analise_subconjunto['Acertos_XGBoost'] = np.nan_to_num(df_analise_subconjunto['Acertos_XGBoost'])

In [ ]:
def converte(x):
    if x == True:
        return 1
    else:
        return 0

In [ ]:
df_analise_subconjunto['Iguais_ambos'] = df_analise_subconjunto['Acertos_SVM'] == df_analise_subconjunto['Acertos_XGBoost']

In [ ]:
df_analise_subconjunto['Acertos_SVM'].value_counts()

In [ ]:
df_analise_subconjunto['Acertos_XGBoost'].value_counts()

In [ ]:
df_analise_subconjunto['Iguais_ambos'] = df_analise_subconjunto['Iguais_ambos'].apply(converte)

In [ ]:
df_analise_subconjunto['Iguais_ambos'] = df_analise_subconjunto[df_analise_subconjunto['Iguais_ambos'] == df_analise_subconjunto['Target']]

In [ ]:
df_analise_subconjunto['Iguais_ambos'].value_counts()

In [ ]:
df_analise_subconjunto['Acertos_SVM'].value_counts()

In [ ]:
df_analise_subconjunto['Acertos_XGBoost'].value_counts()

## Gráfico dos valores de Alpha e Beta para o ponderamento do modelo estatístico bayesiano com o PSO

In [ ]:
 df_media_desvpad_PSO_rand['solucao_x'] = np.clip(df_media_desvpad_PSO_rand['solucao_x'], 0, 1)

In [ ]:
 df_media_desvpad_PSO_rand['solucao_y'] = np.clip(df_media_desvpad_PSO_rand['solucao_y'], 0, 1)

In [ ]:
df_media_desvpad_PSO_rand

In [ ]:
df_media_desvpad_PSO_rand['solucao_x'].rename('alpha', inplace=True)
df_media_desvpad_PSO_rand['solucao_y'].rename('beta', inplace=True)

df_media_desvpad_PSO_rand.rename(columns={'solucao_x':'alpha',
                          'solucao_y':'beta'}, inplace=True)


In [ ]:
df_media_desvpad_PSO_rand[['alpha', 'beta']].plot(figsize=(12,6))
plt.title("Valores de alpha e beta para as execuções do PSO")
plt.savefig('alpha_beta_pso.png')

In [ ]:
df_media_desvpad_PSO_rand['alpha'].mean()

In [ ]:
df_media_desvpad_PSO_rand['beta'].mean()

# Análise Parâmetros XGBoost

In [ ]:
from sklearn import metrics 
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

X_train_svm, X_test_svm, y_train_svm, y_test_svm = aplicaKFoldRand(x_pca_svm, df_bigml_target, np.random.randint(0,10000))
X_train_xgboost, X_test_xgboost, y_train_xgboost, y_test_xgboost = aplicaKFoldRand(x_pca_xgboost, df_bigml_target, np.random.randint(0,10000))
#gb1 = XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8,
#                       colsample_bytree=0.8, objective= 'binary:logistic', nthread=4)
 
gb1 = XGBClassifier(objective= 'binary:logistic', nthread=4)
 
grid_values = {'n_estimators': np.arange(1, 200, 20),'colsample_bytree':np.arange(0.5, 1.1, 0.1), 'max_depth': np.arange(3, 20), 'learning_rate': np.arange(0.01, 0.21, 0.01)}
grid_gb1_acc = GridSearchCV(gb1, param_grid = grid_values,scoring = 'roc_auc', verbose = 2)
grid_gb1_acc.fit(X_train_xgboost, y_train_xgboost)

#Predict values based on new parameters
y_pred_acc = grid_gb1_acc.predict_proba(X_test_xgboost)

fpr, tpr, thresholds = roc_curve(y_test_xgboost, y_pred_acc[:,1])
roc_auc = auc(fpr, tpr)



# New Model Evaluation metrics 
print('AUC Score : ' + str(roc_auc))
print(grid_gb1_acc.best_params_)




# Aplicação do XGBoost com parametrização atualizada junto ao modelo estatístico e PSO

In [ ]:
def aplicaXGBoostOtimizado(X_train, X_test, y_train):
    xg_reg = xgb.XGBClassifier(objective ='binary:logistic', colsample_bytree = 0.9, learning_rate = 1,
                max_depth = 11, alpha = 10, n_estimators = 60)
    xg_reg.fit(X_train,y_train)
    predictionsXGBoost = xg_reg.predict_proba(X_test)
    return predictionsXGBoost

In [ ]:
import SwarmPackagePy
import matplotlib.pyplot as plt
from SwarmPackagePy import testFunctions as tf
from sklearn.naive_bayes import GaussianNB

df_bigml, df_bigml_target = descarregaBaseDados()
scaler = StandardScaler()

scaler.fit(df_bigml)

tempo_rand = []
melhor_rand = []
rands = []
auc_fa = []
auc_svm = []
auc_xgboost = []
convergencia = False
media_aucs = 0
auc_anterior = 0
total_iguais = 0
i = 0
while(not convergencia):
    print(i)
    rand = np.random.randint(0, 1000)
    t1_start = perf_counter()  
    scaled_data = scaler.transform(df_bigml)  
    pca_svm = PCA(n_components = 16)
    pca_svm.fit(scaled_data)
    x_pca_svm = pca_svm.transform(scaled_data)

    pca_xgboost = PCA(n_components=17)
    pca_xgboost.fit(scaled_data)
    x_pca_xgboost = pca_xgboost.transform(scaled_data)
    rands.append(rand)
    X_train_svm, X_test_svm, y_train_svm, y_test_svm = aplicaKFoldRand(x_pca_svm, df_bigml_target, rand)
    X_train_xgboost, X_test_xgboost, y_train_xgboost, y_test_xgboost = aplicaKFoldRand(x_pca_xgboost, df_bigml_target, rand)
    
    predictionsSVM = aplicaSVM(X_train_svm, X_test_svm, y_train_svm)
    predictionsXGBoost = aplicaXGBoostOtimizado(X_train_xgboost, X_test_xgboost, y_train_xgboost)
    
    fprs, tprs, thresholdss = roc_curve(y_test_svm, predictionsSVM[:,1])
    fprx, tprx, thresholdsx = roc_curve(y_test_svm, predictionsXGBoost[:,1])
    
    auc_svm.append(auc(fprs, tprs))
    auc_xgboost.append(auc(fprx, tprx))
    
    vetor_pbest = []
    vetor_acertos = []
    
    alh = pso(40, retorna_acertos, 0, 1, 2, 50, y_test_svm, vetor_pbest, vetor_acertos)
    solucao = alh.get_Gbest()
    svm_alpha = ( ( 1 - predictionsSVM[:,1] ) ** solucao[0] )
    xgboost_beta =  ( ( 1 - predictionsXGBoost[:,1] ) ** solucao[1])
    predicoes = 1 - (svm_alpha*xgboost_beta)
    qtd_acertos = np.subtract(y_test_svm,np.round(predicoes))
    np.count_nonzero(qtd_acertos == 0)/len(y_test_svm) * 100
    fpr, tpr, thresholds = roc_curve(y_test_svm, predicoes)
    auc_fa.append(auc(fpr, tpr))
    
    media_aucs = round(sum(auc_fa)/len(auc_fa), 2)
    if media_aucs == auc_anterior:
        total_iguais = total_iguais + 1
    else:
        total_iguais = 0
    
    if total_iguais == 10:
        convergencia = True
    
    print('media_aucs:{}'.format(media_aucs))
    auc_anterior = media_aucs
    
    arr = np.array(alh.get_agents())
    t1_stop = perf_counter() 
    melhor_rand.append(alh.get_Gbest())
    tempo_rand.append(t1_stop-t1_start)
    
    print(alh.get_Gbest())

In [ ]:
df_media_desvpad_PSO_rand = pd.DataFrame()
df_media_desvpad_PSO_rand['tempo'] = tempo_rand
df_media_desvpad_PSO_rand['solucoes'] = melhor_rand
df_media_desvpad_PSO_rand['rand_values'] = rands
df_media_desvpad_PSO_rand['aucs'] = auc_fa
df_media_desvpad_PSO_rand['auc_svm'] = auc_svm
df_media_desvpad_PSO_rand['auc_xgboost'] = auc_xgboost
df_media_desvpad_PSO_rand[['solucao_x','solucao_y']] = pd.DataFrame(df_media_desvpad_PSO_rand.solucoes.values.tolist(), index= df_media_desvpad_PSO_rand.index)

### Gráfico das AUCs nas execuções 

In [ ]:
import matplotlib as mpl

# create valid markers from mpl.markers
valid_markers = ([item[0] for item in mpl.markers.MarkerStyle.markers.items() if
item[1] is not 'nothing' and not item[1].startswith('tick') and not item[1].startswith('caret')])
df_media_desvpad_PSO_rand.rename(columns = {'aucs':'AUCs modelo estatístico', 'auc_svm':'AUCs SVM', 'auc_xgboost' : 'AUCs XGBoost'}, inplace = True) 

print(df_media_desvpad_PSO_rand['AUCs modelo estatístico'].std())
ax = df_media_desvpad_PSO_rand[['AUCs modelo estatístico', 'AUCs SVM', 'AUCs XGBoost']].plot(figsize=(16,10))


# valid_markers = mpl.markers.MarkerStyle.filled_markers
markers = np.random.choice(valid_markers, df_media_desvpad_PSO_rand.shape[1], replace=False)
for i, line in enumerate(ax.get_lines()):
    line.set_marker(markers[i])

plt.title('AUCs nas execuções até convergência com otimização do XGBoost')
plt.rcParams.update({'font.size': 21})
plt.xlabel('Execução')
plt.ylabel('AUC')


plt.savefig('auc_pso_bigml_otimizacao_xgboost.png')


In [ ]:
alh = pso(40, retorna_acertos, 0, 1, 2, 50, y_test_svm, vetor_pbest, vetor_acertos)
animationFA(alh.get_agents(), retorna_acertos, 0, 1, y_test_svm, sr=True)

In [ ]:
sum(auc_xgboost)/len(auc_xgboost)

In [ ]:
df_media_desvpad_PSO_rand